# Zillow Research Analysis

by: Armun Shakeri

## Overview

This project analyzes 3 bedroom homes within the state of TX to determine which zipcodes 

## Business Problem

## Data Understanding

Import standard packages and data. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("Data/Zip_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")

Obtain information regarding data columns.

In [3]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23404 entries, 0 to 23403
Data columns (total 281 columns):
 #   Column      Dtype  
---  ------      -----  
 0   RegionID    int64  
 1   SizeRank    int64  
 2   RegionName  int64  
 3   RegionType  object 
 4   StateName   object 
 5   State       object 
 6   City        object 
 7   Metro       object 
 8   CountyName  object 
 9   2000-01-31  float64
 10  2000-02-29  float64
 11  2000-03-31  float64
 12  2000-04-30  float64
 13  2000-05-31  float64
 14  2000-06-30  float64
 15  2000-07-31  float64
 16  2000-08-31  float64
 17  2000-09-30  float64
 18  2000-10-31  float64
 19  2000-11-30  float64
 20  2000-12-31  float64
 21  2001-01-31  float64
 22  2001-02-28  float64
 23  2001-03-31  float64
 24  2001-04-30  float64
 25  2001-05-31  float64
 26  2001-06-30  float64
 27  2001-07-31  float64
 28  2001-08-31  float64
 29  2001-09-30  float64
 30  2001-10-31  float64
 31  2001-11-30  float64
 32  2001-12-31  float64
 33  2002-01-31

Display first 5 rows of data.

In [4]:
data.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31
0,91940,0,77449,zip,TX,TX,NaN,"Houston-The Woodlands-Sugar Land, TX",Harris County,96603.0,...,232376.0,236021.0,239693.0,244103.0,249528.0,255561.0,261406.0,266140.0,269363.0,271087.0
1,91982,1,77494,zip,TX,TX,NaN,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,163540.0,...,330128.0,334969.0,339733.0,346806.0,355476.0,365971.0,374539.0,379908.0,381471.0,380327.0
2,93144,2,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,87170.0,...,167760.0,169442.0,171444.0,173448.0,175569.0,178832.0,182377.0,185780.0,188090.0,189856.0
3,62080,3,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,324450.0,...,814606.0,815163.0,817786.0,818496.0,823195.0,827059.0,836542.0,842837.0,849412.0,852484.0
4,62093,4,11385,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,279395.0,...,749033.0,750202.0,754601.0,758353.0,764191.0,766694.0,772804.0,778368.0,783167.0,785138.0


After examining the information regarding this data, we will remove columns 'RegionID', 'SizeRank', 'RegionType', 'StateName', and 'Metro'. These columns are redundant and offer no new information that would be necessary for analysis. 

In [5]:
data = data.drop(['RegionID', 'SizeRank', 'RegionType', 'StateName', 'Metro'], axis = 1)
data.head()

,RegionName,State,City,CountyName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,...,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31
0,77449,TX,NaN,Harris County,96603.0,96618.0,96448.0,96469.0,96525.0,96681.0,...,232376.0,236021.0,239693.0,244103.0,249528.0,255561.0,261406.0,266140.0,269363.0,271087.0
1,77494,TX,NaN,Fort Bend County,163540.0,163790.0,164054.0,164736.0,164592.0,164357.0,...,330128.0,334969.0,339733.0,346806.0,355476.0,365971.0,374539.0,379908.0,381471.0,380327.0
2,79936,TX,El Paso,El Paso County,87170.0,87231.0,87281.0,87299.0,87355.0,87358.0,...,167760.0,169442.0,171444.0,173448.0,175569.0,178832.0,182377.0,185780.0,188090.0,189856.0
3,11368,NY,New York,Queens County,324450.0,328421.0,331099.0,335415.0,338539.0,341713.0,...,814606.0,815163.0,817786.0,818496.0,823195.0,827059.0,836542.0,842837.0,849412.0,852484.0
4,11385,NY,New York,Queens County,279395.0,280318.0,281786.0,283552.0,284726.0,285687.0,...,749033.0,750202.0,754601.0,758353.0,764191.0,766694.0,772804.0,778368.0,783167.0,785138.0


Drop all N/A values. 

In [6]:
data = data.dropna()
data.head()

,RegionName,State,City,CountyName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,...,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31
2,79936,TX,El Paso,El Paso County,87170.0,87231.0,87281.0,87299.0,87355.0,87358.0,...,167760.0,169442.0,171444.0,173448.0,175569.0,178832.0,182377.0,185780.0,188090.0,189856.0
6,60629,IL,Chicago,Cook County,133522.0,133545.0,133889.0,134658.0,135686.0,136885.0,...,247985.0,250548.0,253484.0,255487.0,257701.0,259941.0,263459.0,265572.0,266131.0,265415.0
7,77084,TX,Houston,Harris County,96608.0,96564.0,96319.0,96256.0,96219.0,96379.0,...,227606.0,230914.0,234220.0,237989.0,242657.0,247697.0,252703.0,256619.0,259138.0,260480.0
8,91331,CA,Los Angeles,Los Angeles County,144773.0,145040.0,145795.0,147167.0,148867.0,150796.0,...,659038.0,665242.0,669849.0,677464.0,687363.0,700572.0,711423.0,712829.0,709625.0,698236.0
9,90650,CA,Norwalk,Los Angeles County,172932.0,173000.0,173534.0,174565.0,175941.0,177514.0,...,639038.0,644314.0,649243.0,657427.0,670768.0,687588.0,702447.0,705918.0,704289.0,692281.0


Drop all states outside of TX. 

In [7]:
data = data[data['State'] == 'TX']
data.head(50)

,RegionName,State,City,CountyName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,...,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31
2,79936,TX,El Paso,El Paso County,87170.0,87231.0,87281.0,87299.0,87355.0,87358.0,...,167760.0,169442.0,171444.0,173448.0,175569.0,178832.0,182377.0,185780.0,188090.0,189856.0
7,77084,TX,Houston,Harris County,96608.0,96564.0,96319.0,96256.0,96219.0,96379.0,...,227606.0,230914.0,234220.0,237989.0,242657.0,247697.0,252703.0,256619.0,259138.0,260480.0
17,78660,TX,Pflugerville,Travis County,142867.0,143284.0,143433.0,143843.0,144089.0,144397.0,...,404382.0,415007.0,424494.0,435592.0,446426.0,455652.0,462620.0,458565.0,449787.0,438317.0
34,75052,TX,Grand Prairie,Dallas County,103839.0,103974.0,104096.0,104320.0,104531.0,104727.0,...,278604.0,282369.0,287535.0,294913.0,302589.0,311724.0,319510.0,326135.0,328404.0,327660.0
36,77433,TX,Cypress,Harris County,125573.0,125633.0,125507.0,125746.0,125928.0,126252.0,...,276447.0,280418.0,284331.0,289139.0,295270.0,302052.0,308682.0,314240.0,318226.0,320487.0
43,77479,TX,Sugar Land,Fort Bend County,159504.0,159921.0,160374.0,161523.0,161895.0,161914.0,...,326401.0,330105.0,333949.0,340363.0,348369.0,357723.0,365308.0,369758.0,370600.0,368421.0
52,77429,TX,Cypress,Harris County,124622.0,124617.0,124414.0,124533.0,124652.0,124959.0,...,275276.0,279240.0,283230.0,288011.0,294239.0,300911.0,307449.0,312895.0,316762.0,319017.0
57,75217,TX,Dallas,Dallas County,70802.0,70910.0,70898.0,71028.0,71141.0,71319.0,...,195299.0,198111.0,202856.0,208669.0,214852.0,220711.0,225957.0,231034.0,233564.0,233783.0
60,77584,TX,Pearland,Brazoria County,138642.0,138983.0,139295.0,139948.0,140671.0,141079.0,...,283008.0,286918.0,290409.0,294671.0,300091.0,305651.0,311534.0,315514.0,318971.0,321427.0
74,77379,TX,Spring,Harris County,124664.0,124584.0,124321.0,124315.0,124400.0,124789.0,...,255049.0,258572.0,262074.0,265930.0,270820.0,276161.0,281865.0,286218.0,289087.0,290343.0


Since zipcodes are hierarchial we will sort values in 'RegionName' by ascending order. 

In [8]:
data.sort_values('RegionName', ascending=True).head()

,RegionName,State,City,CountyName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,...,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31
7529,75001,TX,Addison,Dallas County,222199.0,222363.0,221914.0,221323.0,221022.0,220851.0,...,457879.0,460682.0,466261.0,475972.0,486368.0,501219.0,512714.0,524172.0,528349.0,530920.0
6689,75009,TX,Celina,Collin County,161634.0,160980.0,161030.0,159987.0,158775.0,158109.0,...,411317.0,423242.0,436911.0,449736.0,460649.0,472905.0,485917.0,495647.0,496439.0,492622.0
1545,75013,TX,Allen,Collin County,217534.0,216980.0,217051.0,216918.0,217344.0,217217.0,...,453690.0,465545.0,480324.0,493147.0,503993.0,517230.0,530523.0,541898.0,542785.0,538287.0
1218,75019,TX,Coppell,Dallas County,174581.0,174785.0,175122.0,175810.0,176304.0,176267.0,...,409922.0,413336.0,419358.0,428912.0,438896.0,452717.0,464373.0,475268.0,479038.0,477770.0
4245,75022,TX,Flower Mound,Denton County,216089.0,216549.0,217381.0,217894.0,218790.0,218108.0,...,513792.0,524956.0,539656.0,554848.0,568416.0,581669.0,593180.0,601686.0,601160.0,597419.0


Next we will need to filter our date ranges, we will only want data for years 2018-2022(present).

In [10]:
data.load()

AttributeError: 'DataFrame' object has no attribute 'load'